# Section 1, Data Preparation 01
### by Xitian Sun
#### this file is aimed to use the structed datasets from MIMIC-iv 2.2 into suitable forms for machine learning
 inovated by  Med-BERT and ExBEHRT proposed by previous papers, here are some ideas of how to construct our own traning set;

## the new data forms must meet following requirements:
 the data will be made up with records of different individual patients;

 the records of individual patients should be made up by different visits;

 the visits should be including diagnosis codes(information of their disease information), time stamp of visits, other interested related information with accordance with the visits;
 

In [2]:
# package importing section
# basic numeric operation packages
import pandas as pd
import numpy as np
import random
from datetime import datetime as dt
from datetime import timedelta
import sys
from optparse import OptionParser

try:
    import cPickle as pickle
except:
    import pickle
#package required for tensors for training

In [2]:
#data loading for Elvis
admission =     pd.read_csv(r"F:\SchoolWork\research\Med-BERT\datasets\admissions.csv")
diagnoses_icd = pd.read_csv(r"F:\SchoolWork\research\Med-BERT/datasets/diagnoses_icd.csv")
drgcodes =      pd.read_csv(r"F:\SchoolWork/research/Med-BERT\datasets\drgcodes.csv")
procedures_icd = pd.read_csv(r"F:\SchoolWork/research/Med-BERT\datasets\procedures_icd.csv")
#procedures_icd = pd.read_csv(r"F:\SchoolWork/research/Med-BERT\datasets\procedures_icd.csv")
gem_df = pd.read_csv(r"F:\SchoolWork\research\Med-BERT\datasets\icd9toicd10cmgem.csv")
patients = pd.read_csv(r"F:\SchoolWork\research\Med-BERT\datasets\patients.csv")
gempc_df = pd.read_csv(r"F:\SchoolWork\research\Med-BERT\datasets\icd9toicd10pcsgem.csv",dtype={'icd9cm': str})

In [3]:
admission.head(5)
#drgcodes.head(5)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23,2180-05-07 17:15,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17,2180-05-06 23:30,0
1,10000032,22841357,2180-06-26 18:27,2180-06-27 18:49,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54,2180-06-26 21:31,0
2,10000032,25742920,2180-08-05 23:44,2180-08-07 17:50,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58,2180-08-06 1:44,0
3,10000032,29079034,2180-07-23 12:35,2180-07-25 17:55,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 5:54,2180-07-23 14:00,0
4,10000068,25022803,2160-03-03 23:16,2160-03-04 6:26,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55,2160-03-04 6:26,0


In [4]:

#mapping ICD-9 TO ICD-10
#em_df = pd.read_csv(r"F:\SchoolWork\research\Med-BERT\datasets\icd9toicd10cmgem.csv")
print(gem_df.head())
icd9_to_icd10 = dict(zip(gem_df['icd9cm'], gem_df['icd10cm']))

icd9_code = '0010' 
icd10_code = icd9_to_icd10.get(icd9_code, 'MAPPING UNFOUND')

print(f'ICD-9 codes {icd9_code} the corresponding icd-10 code is : {icd10_code}')
diagnoses_icd.loc[diagnoses_icd['icd_version'] == 9, 'icd_code'] = diagnoses_icd.loc[diagnoses_icd['icd_version'] == 9, 'icd_code'].apply(lambda x: icd9_to_icd10.get(x, 'MAPPING UNFOUND'))
diagnoses_icd['icd_code'] = diagnoses_icd['icd_code'].str.slice(0, 3)
diagnoses_icd.head(30)


  icd9cm icd10cm  flags  approximate  no_map  combination  scenario  \
0   0010    A000      0            0       0            0         0   
1   0011    A001      0            0       0            0         0   
2   0019    A009      0            0       0            0         0   
3   0020   A0100  10000            1       0            0         0   
4   0021    A011      0            0       0            0         0   

   choice_list  
0            0  
1            0  
2            0  
3            0  
4            0  
ICD-9 codes 0010 the corresponding icd-10 code is : A000


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,K76,9
1,10000032,22595853,2,R18,9
2,10000032,22595853,3,K74,9
3,10000032,22595853,4,B19,9
4,10000032,22595853,5,J44,9
5,10000032,22595853,6,F31,9
6,10000032,22595853,7,F43,9
7,10000032,22595853,8,Z87,9
8,10000032,22841357,1,B19,9
9,10000032,22841357,2,R18,9


In [5]:
#procedures_icd = pd.read_csv(r"F:\SchoolWork/research/Med-BERT\datasets\procedures_icd.csv")
#gempc_df = pd.read_csv(r"F:\SchoolWork\research\Med-BERT\datasets\icd9toicd10pcsgem.csv",dtype={'icd9cm': str})
print(gempc_df.head())
icd9_to_icd10pc = dict(zip(gempc_df['icd9cm'], gempc_df['icd10cm']))
#procedures_icd.head(30)

  icd9cm  icd10cm  flags  approximate  no_map  combination  scenario  \
0   0001  6A750Z4  10000            1       0            0         0   
1   0001  6A751Z4  10000            1       0            0         0   
2   0002  6A750Z5  10000            1       0            0         0   
3   0002  6A751Z5  10000            1       0            0         0   
4   0003  6A750Z6  10000            1       0            0         0   

   choice_list  
0            0  
1            0  
2            0  
3            0  
4            0  


In [6]:
icd9_code = '7569' 
icd10_code = icd9_to_icd10pc.get(icd9_code, 'MAPPING UNFOUND')

print(f'ICD-9 codes {icd9_code} the corresponding icd-10 code is : {icd10_code}')
procedures_icd.loc[procedures_icd['icd_version'] == 9, 'icd_code'] = procedures_icd.loc[procedures_icd['icd_version'] == 9, 'icd_code'].apply(lambda x: icd9_to_icd10pc.get(x, 'MAPPING UNFOUND'))
procedures_icd.head()

ICD-9 codes 7569 the corresponding icd-10 code is : 0WQNXZZ


,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10000032,22595853,1,2180-05-07,0W9J3ZZ,9
1,10000032,22841357,1,2180-06-27,0W9J3ZZ,9
2,10000032,25742920,1,2180-08-06,0W9J3ZZ,9
3,10000068,25022803,1,2160-03-03,4A09XMZ,9
4,10000117,27988844,1,2183-09-19,0QS734Z,10


In [7]:
hadm_list = pd.unique(admission['hadm_id'])
admission.rename(columns={
    'subject_id': 'patient_sk',
    'admittime': 'admit_dt_tm',
    'dischtime': 'discharge_dt_tm',
    'insurance': 'third_party_ind'
}, inplace=True)

patients.rename(columns={
    'subject_id': 'patient_sk',
}, inplace=True)

diagnoses_icd.rename(columns={
    'icd_code': 'diagnosis'
}, inplace=True)

procedures_icd.rename(columns={
    'icd_code': 'procedure'
}, inplace=True)

merged_data0 = pd.merge(admission, diagnoses_icd, on='hadm_id', how='left')
merged_data1 = pd.merge(merged_data0 ,drgcodes, on='hadm_id', how='left') 
merged_data2 = pd.merge(merged_data1 ,procedures_icd, on='hadm_id', how='left')
merged_data = pd.merge(merged_data2 ,patients, on='patient_sk', how='left')

merged_data['drg_code'] = merged_data['drg_type'].str.cat(merged_data['drg_code'].astype(str).str.slice(0, 3), sep='')
final_data = merged_data[['patient_sk', 'race','language','marital_status','admit_dt_tm', 'discharge_dt_tm', 'third_party_ind', 'diagnosis','drg_code','procedure','gender']]

In [8]:
final_data.head()

,patient_sk,race,language,marital_status,admit_dt_tm,discharge_dt_tm,third_party_ind,diagnosis,drg_code,procedure
0,10000032,WHITE,ENGLISH,WIDOWED,2180-05-06 22:23,2180-05-07 17:15,Other,K76,APR283,0W9J3ZZ
1,10000032,WHITE,ENGLISH,WIDOWED,2180-05-06 22:23,2180-05-07 17:15,Other,K76,HCFA442,0W9J3ZZ
2,10000032,WHITE,ENGLISH,WIDOWED,2180-05-06 22:23,2180-05-07 17:15,Other,R18,APR283,0W9J3ZZ
3,10000032,WHITE,ENGLISH,WIDOWED,2180-05-06 22:23,2180-05-07 17:15,Other,R18,HCFA442,0W9J3ZZ
4,10000032,WHITE,ENGLISH,WIDOWED,2180-05-06 22:23,2180-05-07 17:15,Other,K74,APR283,0W9J3ZZ


In [9]:
final_data['admit_dt_tm'] = final_data['admit_dt_tm'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M").strftime("%Y-%m-%d"))
final_data['discharge_dt_tm'] = final_data['discharge_dt_tm'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M").strftime("%Y-%m-%d"))

c:\Users\xitia\anaconda3\envs\medbert_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\xitia\anaconda3\envs\medbert_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:

final_data.to_csv(r'F:\SchoolWork\research\trial code\DATA695-2024-05\BERTs\codesdiagdatashort.csv', index=False)

print(final_data.head())

   patient_sk   race language marital_status admit_dt_tm discharge_dt_tm  \
0    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
1    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
2    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
3    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
4    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   

  third_party_ind diagnosis drg_code procedure  
0           Other       K76   APR283   0W9J3ZZ  
1           Other       K76  HCFA442   0W9J3ZZ  
2           Other       R18   APR283   0W9J3ZZ  
3           Other       R18  HCFA442   0W9J3ZZ  
4           Other       K74   APR283   0W9J3ZZ  


In [3]:
# data should be saved locally

# Section 1, Data Preparation 02
## there are several target we are trying to realize in this section:
### based on the previously merged and standardlized dataset, we want to 

In [3]:
data_diag = pd.read_csv(r'F:\SchoolWork\research\trial code\DATA695-2024-05\BERTs\codesdiagdatashort.csv')

In [5]:
data_diag.columns

Index(['patient_sk', 'race', 'language', 'marital_status', 'admit_dt_tm',
       'discharge_dt_tm', 'third_party_ind', 'diagnosis', 'drg_code',
       'procedure', 'gender'],
      dtype='object')

In [4]:
import pickle
import pandas as pd

def read_typeFile(data_diag,typeFile, p_samplesize):
    if typeFile == 'NA':
        types = {'empty_pad': 0}
    else:
        with open(typeFile, 'rb') as t2:
            types = pickle.load(t2)

    if p_samplesize > 0:
        print('Sampling')
        ptsk_list = data_diag['patient_sk'].drop_duplicates()
        pt_list_samp = ptsk_list.sample(n=p_samplesize)
        n_data = data_diag[data_diag["patient_sk"].isin(pt_list_samp.values.tolist())]
    else:
        n_data = data_diag

    return types, n_data

print('Start Data Preprocessing !!!')


Start Data Preprocessing !!!


In [58]:
#for the whole dataset to be processed, the p_samplesize should be set to 0
types, n_data=read_typeFile(data_diag,typeFile='NA', p_samplesize=100000)



Sampling


In [59]:
print(n_data.head(10))

   patient_sk   race language marital_status admit_dt_tm discharge_dt_tm  \
0    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
1    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
2    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
3    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
4    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
5    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
6    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
7    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
8    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   
9    10000032  WHITE  ENGLISH        WIDOWED  2180-05-06      2180-05-07   

  third_party_ind diagnosis drg_code procedure gender  
0           Other       K76   APR283   0W9J3ZZ      F  
1           Other       K76  HCFA442   0W9J3ZZ     

In [60]:
import pickle
def split_fn( pts_sls, outFile,record_type ='ptencs' ):
    #record_type can also be 'bencs'
    print("Splitting")
    dataSize = len(pts_sls)
    np.random.seed(0)
    ind = np.random.permutation(dataSize)
    nTest = int(0.2 * dataSize)
    nValid = int(0.2 * dataSize)
    test_indices = ind[:nTest]
    valid_indices = ind[nTest:nTest + nValid]
    train_indices = ind[nTest + nValid:]

    for subset in ['train', 'valid', 'test']:
        if subset == 'train':
            indices = train_indices
        elif subset == 'valid':
            indices = valid_indices
        elif subset == 'test':
            indices = test_indices
        else:
            print('error')
            break
        subset_ptencs = [pts_sls[i] for i in indices]   #pts_ls contains single records of single visits 
        #subset_ptencs_s = [pts_sls[i] for i in indices]#pts_sls contains records of a patients of multiple visit
        ptencsfile = outFile + '.'+record_type+"." + subset
        #bertencsfile = outFile + '.bencs.' + subset
        #serialization
        pickle.dump(subset_ptencs, open(ptencsfile, 'a+b'), -1)  # turn data file converted into a byte stream
        #pickle.dump(subset_ptencs_s, open(bertencsfile, 'a+b'), -1)

In [61]:
#initialization of variables
outFile = "F:/SchoolWork/research/trial code/DATA695-2024-05/BERTs/datasetsdataprocessed"
count = 0
types = {'empty_pad': 0}
typesdrg = {'empty_pad': 0}
typespcd = {'empty_pad': 0}
typesrace = {'empty_pad': 0}
typeslan = {'empty_pad': 0}
typesms = {'empty_pad': 0}
typegender={'empty_pad': 0}
pts_ls = []
pts_sls = []
for Pt, group in n_data.groupby('patient_sk'):
        pt_encs = []
        time_tonext = []
        pt_los = []
        full_seq_diag = []
        full_seq_drg = []
        full_seq_pcd = []
        v_seg = []
        pt_discdt = []
        pt_addt = []
        pt_ls = []
        pt_drg=[]
        pt_pcd=[]
        pt_ms=[]
        pt_lan=[]
        pt_race=[]
        pt_gender = []
        v = 0
        #recording admission and discharge time(calcualtion of LOS and TimetoNext)
        for Time, subgroup in group.sort_values(['discharge_dt_tm'], ascending=True).groupby('discharge_dt_tm', sort=False):
            v = v + 1
            diag_l = np.array(subgroup['diagnosis'].drop_duplicates()).tolist()
            drg_l = np.array(subgroup['drg_code'].drop_duplicates()).tolist()
            pcd_l = np.array(subgroup['procedure'].drop_duplicates()).tolist()
            race_l=np.array(subgroup['race'].drop_duplicates()).tolist()
            lan_l=np.array(subgroup['language'].drop_duplicates()).tolist()
            ms_l = np.array(subgroup['marital_status'].drop_duplicates()).tolist()
            gender_l = np.array(subgroup['gender'].drop_duplicates()).tolist()

        if len(diag_l) > 0:
            diag_lm = []
            for code in diag_l:
                if code in types:
                    diag_lm.append(types[code])
                else:
                    types[code] = max(types.values()) + 1
                    diag_lm.append(types[code])

        if len(drg_l) > 0:
            drg_lm = []
            for code in drg_l:
                typesdrg[code] = max(typesdrg.values()) + 1
                drg_lm.append(typesdrg[code])

        if len(pcd_l) > 0:
            pcd_lm = []
            for code in pcd_l:
                typespcd[code] = max(typespcd.values()) + 1       
                pcd_lm.append(typespcd[code])

        if len(race_l) > 0:
            race_lm = []
            for code in race_l:
                typesrace[code] = max(typesrace.values()) + 1 
                race_lm.append(typesrace[code])

        if len(ms_l) > 0:
            ms_lm = []
            for code in ms_l:
                typesms[code] = max(typesms.values()) + 1 
                ms_lm.append(typesms[code])

        if len(lan_l) > 0:
            lan_lm = []
            for code in lan_l:
                typeslan[code] = max(typeslan.values()) + 1
                lan_lm.append(typeslan[code])

        if len(gender_l) > 0:
            gender_lm = []
            for code in gender_l:
                if code in typegender:
                    gender_lm.append(typegender[code])
                else:
                    typegender[code] = max(typegender.values()) + 1
                    gender_lm.append(typegender[code])

        
        #print(pt_gender)
        v_seg.append(v)



        full_seq_diag.extend(diag_lm)
        full_seq_drg.extend(drg_lm)
        full_seq_pcd.extend(pcd_lm)

        pt_discdt.append((dt.strptime(Time, '%Y-%m-%d')))
        pt_addt.append(dt.strptime(min(np.array(subgroup['admit_dt_tm'].drop_duplicates()).tolist()), '%Y-%m-%d'))

        if len(pt_discdt) > 0:
            for ei, eid in enumerate(pt_discdt):
                if ei == len(pt_discdt) - 1:
                    enc_td = 0
                else:
                    enc_td = (pt_addt[ei + 1] - pt_discdt[ei]).days # time gap from this time to next visit to hospital
                enc_los = (pt_discdt[ei] - pt_addt[ei]).days #stay of length in hospital#
                time_tonext.append(enc_td)
                pt_los.append(enc_los)

                enc_l = [gender_lm,enc_los, enc_td, diag_lm,drg_lm,pcd_lm,ms_lm,lan_lm,race_lm,v]
                pt_encs.append(enc_l)

        if len(pt_encs) > 0:
            pt_ls.append(pt_encs)



        
        pts_ls.append(str(pt_ls))
        pts_sls.append(str([gender_lm,pt_los, time_tonext,full_seq_diag,full_seq_drg, full_seq_pcd, ms_lm,lan_lm,race_lm,v_seg]))
        count = count + 1

       # if count % 1000 == 0:
       #     print('Processed %d pts' % count)

       # if count % 100000 == 0:
        #    print('Dumping %d pts' % count)
        #    split_fn(pts_ls, pts_sls, outFile)
        #    pts_ls = []
        #    pts_sls = []
#pts_ls = ' '.join([str(x) for x in pts_ls])
#pts_sls = ' '.join([str(x) for x in pts_sls])



In [62]:
pts_ls

['[[[[1], 2, 0, [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2], [1], [1], [1], [1], 4]]]',
 '[[[[2], 1, 0, [10, 11], [3], [2], [2], [2], [2], 1]]]',
 '[[[[2], 0, 0, [12, 13, 14, 15, 16, 17], [4], [3], [3], [3], [3], 1]]]',
 '[[[[2], 0, 0, [18], [5], [4], [4], [4], [4], 1]]]',
 '[[[[1], 2, 0, [19, 20, 21], [6, 7], [5], [5], [5], [5], 1]]]',
 '[[[[1], 1, 0, [22, 23, 24], [8], [6, 7, 8], [6], [6], [6], 1]]]',
 '[[[[1], 3, 0, [25, 26, 27, 28, 29], [9, 10], [9], [7], [7], [7], 1]]]',
 '[[[[2], 8, 0, [30, 31], [11, 12], [10], [8], [8], [8], 2]]]',
 '[[[[2], 0, 0, [32], [13], [11], [9], [9], [9], 1]]]',
 '[[[[1], 1, 0, [33], [14], [12], [10], [10], [10], 1]]]',
 '[[[[1], 16, 0, [34, 35, 36, 37, 38, 22, 39, 40, 41, 42, 5, 43, 44, 45, 46, 20, 47, 48, 49, 50, 51, 52], [15, 16], [13], [11], [11], [11], 5]]]',
 '[[[[1], 2, 0, [49, 17, 23, 53, 54, 55, 56, 5, 57, 58, 59, 21, 60, 61, 62, 63, 64, 29], [17, 18], [14], [12], [12], [12], 7]]]',
 '[[[[1], 3, 0, [65, 9, 66, 67, 68, 69, 70, 53, 17, 49, 24, 5, 71, 23, 

In [63]:
type( pts_ls)

list

In [64]:
import os
print("Current working directory:", os.getcwd())

Current working directory: f:\SchoolWork\research\trial code\DATA695-2024-05\DATA695-2024-05.worktrees\Elvis's-exploration


In [65]:
split_fn(pts_ls, outFile)
pickle.dump(types, open(outFile + '.types', 'wb'), -1)
pickle.dump(typeslan, open(outFile + '.typeslan', 'wb'), -1)
pickle.dump(typesms, open(outFile + '.typesms', 'wb'), -1)
pickle.dump(typesrace, open(outFile + '.typesrace', 'wb'), -1)
pickle.dump(typesdrg, open(outFile + '.typesdrg', 'wb'), -1)
pickle.dump(typespcd, open(outFile + '.typespcd', 'wb'), -1)
pickle.dump(typegender, open(outFile + '.typesgender', 'wb'), -1)


Splitting


# Part3 Model Pretraining

In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
#import tensorflow_models as tfm
import tensorflow_hub as hub
import tensorflow_text
#import tensorflow_datasets as tfds

In [3]:
# To verify if we have GPU on our local machine.if we have, then how many?
if tf.config.experimental.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

GPU is available
Num GPUs Available:  1


In [4]:
import pickle

#input_file = tf.io.gfile.glob(input)
def load_data(input_url,  dtype):
 input_train = input_url +'.'+ dtype+'.'+"train"
 with open(input_train, 'rb') as ptencs_f:
            while True:
                try:
                    #print(ptencs_f)
                    input_train=pickle.load(ptencs_f)
                    #subset_ptencs.append(pickle.load(ptencs_f))
                except EOFError:
                    break
 del ptencs_f

 input_valid = input_url +'.'+ dtype+'.'+"valid"
 with open(input_valid, 'rb') as ptencs_f:
            while True:
                try:
                    #print(ptencs_f)
                    input_valid=pickle.load(ptencs_f)
                    #subset_ptencs.append(pickle.load(ptencs_f))
                except EOFError:
                    break
 del ptencs_f

 input_test = input_url +'.'+ dtype+'.'+"test"
 with open(input_test, 'rb') as ptencs_f:
            while True:
                try:
                    #print(ptencs_f)
                    input_test=pickle.load(ptencs_f)
                    #subset_ptencs.append(pickle.load(ptencs_f))
                except EOFError:
                    break
 del ptencs_f

 return input_train,input_valid,input_test

In [5]:
inFile = 'F:/SchoolWork/research/trial code/DATA695-2024-05/BERTs/datasetsdataprocessed'
dtype = 'ptencs'
input_train,input_valid,input_test = load_data(inFile,  dtype)

In [7]:
input_train

['[[[[2], 1, 0, [153, 99, 23, 48, 56, 247, 57, 20, 100, 49, 41, 59, 53, 5, 246, 108, 110, 104, 158, 106, 313, 244, 58], [137040, 137041], [158853, 158854, 158855, 158856, 158857, 158858], [81137], [81137], [81137], 1]]]',
 '[[[[2], 1, 0, [53, 22, 61, 1231, 101], [134778], [156181], [79805], [79805], [79805], 1]]]',
 '[[[[1], 0, 0, [315, 31, 295, 35, 482, 107], [58950], [68326], [34904], [34904], [34904], 3]]]',
 '[[[[2], 1, 0, [267, 22, 266, 24, 23, 8], [107657], [124995], [63753], [63753], [63753], 1]]]',
 '[[[[1], 10, 0, [329, 394, 62, 22, 78, 672, 193, 15, 96, 440, 119, 55, 666, 231, 60, 239, 168, 111, 478, 37, 503, 61, 286, 80, 70, 41, 185], [86338, 86339], [99939, 99940, 99941], [51130], [51130], [51130], 1]]]',
 '[[[[1], 0, 0, [105, 280, 122, 90, 150, 798, 85, 24, 23, 82], [48661], [56356], [28795], [28795], [28795], 1]]]',
 '[[[[2], 3, 0, [56, 334, 138, 31, 49, 23, 125, 158, 40, 348, 5, 15, 305, 61], [111065], [128923], [65759], [65759], [65759], 2]]]',
 '[[[[2], 5, 0, [503, 24,

In [10]:
map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_uncased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/3',
    'bert_en_wwm_uncased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_en_cased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-24_H-1024_A-16/3',
    'bert_en_wwm_cased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_wwm_cased_L-24_H-1024_A-16/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'albert_en_large':
        'https://tfhub.dev/tensorflow/albert_en_large/2',
    'albert_en_xlarge':
        'https://tfhub.dev/tensorflow/albert_en_xlarge/2',
    'albert_en_xxlarge':
        'https://tfhub.dev/tensorflow/albert_en_xxlarge/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
    'talking-heads_large':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_large/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_wwm_cased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'bert_en_cased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'bert_en_wwm_uncased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'albert_en_large':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'albert_en_xlarge':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'albert_en_xxlarge':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_large':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}



In [11]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print('BERT model selected           :', tfhub_handle_encoder)
print('Preprocessing model auto-selected:', tfhub_handle_preprocess)

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocessing model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [12]:
def make_bert_preprocess_model(sentence_features, seq_length=128):
  """Returns Model mapping string features to BERT inputs.

  Args:
    sentence_features: a list with the names of string-valued features.
    seq_length: an integer that defines the sequence length of BERT inputs.

  Returns:
    A Keras Model that can be called on a list or dict of string Tensors
    (with the order or names, resp., given by sentence_features) and
    returns a dict of tensors for input to BERT.
  """

  input_segments = [
      tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
      for ft in sentence_features]

  # Tokenize the text to word pieces.
  bert_preprocess = hub.load(tfhub_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in input_segments]

  # Optional: Trim segments in a smart way to fit seq_length.
  # Simple cases (like this example) can skip this step and let
  # the next step apply a default truncation to approximately equal lengths.
  truncated_segments = segments

  # Pack inputs. The details (start/end token ids, dict of output tensors)
  # are model-dependent, so this gets loaded from the SavedModel.
  packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
                          arguments=dict(seq_length=seq_length),
                          name='packer')
  model_inputs = packer(truncated_segments)
  return tf.keras.Model(input_segments, model_inputs)

In [ ]:
test_preprocess_model = make_bert_preprocess_model(['my_input1', 'my_input2'])

In [31]:

test_text = [np.array(['some random test sentence']),
             np.array(['another sentence'])]
text_preprocessed = test_preprocess_model(test_text)

print('Keys           : ', list(text_preprocessed.keys()))
print('Shape Word Ids : ', text_preprocessed['input_word_ids'].shape)
print('Word Ids       : ', text_preprocessed['input_word_ids'][0, :16])
print('Shape Mask     : ', text_preprocessed['input_mask'].shape)
print('Input Mask     : ', text_preprocessed['input_mask'][0, :16])
print('Shape Type Ids : ', text_preprocessed['input_type_ids'].shape)
print('Type Ids       : ', text_preprocessed['input_type_ids'][0, :16])

Keys           :  ['input_type_ids', 'input_word_ids', 'input_mask']
Shape Word Ids :  (1, 128)
Word Ids       :  tf.Tensor(
[ 101 2070 6721 3231 6251  102 2178 6251  102    0    0    0    0    0
    0    0], shape=(16,), dtype=int32)
Shape Mask     :  (1, 128)
Input Mask     :  tf.Tensor([1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0], shape=(16,), dtype=int32)
Shape Type Ids :  (1, 128)
Type Ids       :  tf.Tensor([0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0], shape=(16,), dtype=int32)


In [17]:
print(tf.__version__)
print(hub.__version__)


2.8.4
0.12.0


In [ ]:
#train_preprocess_model =make_bert_preprocess_model(input_train)

In [15]:
AUTOTUNE = tf.data.AUTOTUNE


def load_dataset_from_tfds(in_memory_ds, info, split, batch_size,
                           bert_preprocess_model):
  is_training = split.startswith('train')
  dataset = tf.data.Dataset.from_tensor_slices(in_memory_ds[split])
  num_examples = info.splits[split].num_examples

  if is_training:
    dataset = dataset.shuffle(num_examples)
    dataset = dataset.repeat()
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex['label']))
  dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
  return dataset, num_examples

In [ ]:
def build_classifier_model(num_classes):

  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
test_classifier_model = build_classifier_model(2)
bert_raw_result = test_classifier_model(text_preprocessed)
print(tf.sigmoid(bert_raw_result))